In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import os
import re
import matplotlib.colors as mcolors
from scipy.stats import binom_test


# Define function with two optional parameters; base name of file and output directory (default is current directory)
def get_next_version(base_name="version", output_dir="."):
    """Helper function to get the next version number when starting a new run"""
    # Retrieve list of all filenames in directory
    files = os.listdir(output_dir)
    # Regular expression to match filenames in format version_XXX_plotX.pdf
    pattern = re.compile(rf"{base_name}_(\d+)_plot\d+\.pdf")
    versions = []
    
    # Iterates over each file in files list to look for matching filenames
    for file in files:
        match = pattern.match(file)
        if match:
            version_num = int(match.group(1))
            versions.append(version_num)
            
    # If files already exist in directory, find the highest version number and increment by 1
    if versions:
        versions.sort()
        latest_version = versions[-1]
        next_version = latest_version + 1
        return f"{next_version:03d}"  # Format with leading zeros (e.g., 001, 002)
    return "001" 

def plot_poll_results(df, title_prefix="Plot"):
    # Retrieve poll IDs from Poll ID column
    poll_ids = df["Poll ID"].unique()

    # Define grey shades for each subplot
    poll_colors = ['#606060', '#606060', '#606060']  # Greys for all bars
    
    # Extract the title; first value from Poll Question column
    title_text = df["Poll Question"].iloc[0]

    # Extract the number and the main text
    result = re.match(r"(\d+)[a-zA-Z]*\.?\s.*?Please grade the VALIDITY of (.+?)\s*\*.*", title_text) # Code to extract title for initial scoring
    #result = re.match(r"(\d+(?:\.\d+)*)\.?[a-zA-Z]*\.?\s.*?Please grade the VALIDITY of (.*)", title_text) # Code to extract title for regrade

    # Format the extracted poll number and main question text
    if result:
        number = result.group(1)
        main_text = result.group(2).strip().capitalize()
        cleaned_text = f"{number}. {main_text}"

    # Create a figure and axes
    fig, axes = plt.subplots(1, len(poll_ids), figsize=(21, 12))
    
    # Iterate through each poll ID and extract corresponding data
    for i, poll_id in enumerate(poll_ids):
        subset = df[df["Poll ID"] == poll_id]
        
        subset_nonzero = subset[subset["Count"] > 0]  # Filter for non-zero counts
        
        # Set background color for each subplot
        axes[i].set_facecolor('#E6EEF7') # Light blue

        # Extract numeric value from poll option column, accounting for columns with string values
        subset_nonzero["Poll Option Numeric"] = subset_nonzero["Poll Option"].str.split(',').str[0].astype(int)

        # Extract and set first keyword after "Please grade the" as title of each subplot (Validity, Importance, etc.)
        key_word = subset["Poll Question"].iloc[0].split("Please grade the ")[1].split()[0].capitalize()
        axes[i].set_title(key_word, fontsize=21, pad = 16)

        # Expand poll option values based on their counts and calclulates median
        expanded_values = np.repeat(subset_nonzero["Poll Option Numeric"], subset_nonzero["Count"])
        median = np.median(expanded_values)

        max_count = subset["Count"].max() + 1
        
        # Create y-axis values (1 through 9) and assign bar colors based on poll counts
        y_values = list(range(1, 10))
        bar_colors = ['white' if count == 0 else poll_colors[i] for count in subset["Count"]]
        
        # Plots horizontal bars with assigned colors and transparency
        axes[i].barh(y_values, subset["Count"], color=bar_colors, alpha=0.8, height=0.72)

        # Add numeric labels to bars
        for y, count in zip(y_values, subset["Count"]):
            if count > 0:
                axes[i].text(count + 0.1, y, str(count), va='center', fontsize=15)
    
        if i == 0:
            if max_count >= 8:
                axes[i].text(-1, 1, "Not", fontsize=18.5, ha='center', va='center')
                axes[i].text(-2, 5, "Somewhat", fontsize=18.5, ha='center', va='center')
                axes[i].text(-1, 9, "Very", fontsize=18.5, ha='center', va='center')
            else:
                axes[i].text(-0.65, 1, "Not", fontsize=18.5, ha='center', va='center')
                axes[i].text(-1.15, 5, "Somewhat", fontsize=18.5, ha='center', va='center')
                axes[i].text(-0.65, 9, "Very", fontsize=18.5, ha='center', va='center')

        axes[i].set_yticks(y_values)
        axes[i].set_yticklabels([str(y) for y in y_values], fontsize=14)

        for label in axes[i].get_yticklabels():
            label.set_fontweight('bold')
            
        # Keep spines visible
        for spine in axes[i].spines.values():
            spine.set_visible(True)
            spine.set_linewidth(1.5)

        # Remove x-ticks
        axes[i].set_xticks([])

        # Set y-tick length to 0 and invert y-axis
        axes[i].tick_params(axis='y', length=0)
        axes[i].invert_yaxis()

        # Function to determine if the color is light or dark for median box 
        def is_light_color(color):
            # Extract RGB values using matplotlib.colors.to_rgb
            r, g, b = mcolors.to_rgb(color)
            # Calculate luminance (perceived brightness)
            luminance = 0.299 * r + 0.587 * g + 0.114 * b
            # Return True if light (luminance > 0.5), else False
            return luminance > 0.5
        
        # Add color-coded median box
        if median <= 3:
            color = 'orange'
        elif median <= 6:
            color = 'yellow'
        else:
            color = 'green'

        def calculate_disagreement_index(subset_nonzero):
            # Expand the data based on the count for percentile calculation
            expanded_data = np.repeat(subset_nonzero["Poll Option Numeric"], subset_nonzero["Count"])

            # Calculate IQR (not necessary for disagreement calculation)
            q75, q25 = np.percentile(expanded_data, [75 ,25])
            iqr = q75 - q25
            
            # Calculate the 30th and 70th percentiles
            IPRL = np.percentile(expanded_data, 30)  # 30th percentile
            IPRU = np.percentile(expanded_data, 70)  # 70th percentile
            
            # Calculate the central point of the IPR (IPRC)
            IPRC = (IPRL + IPRU) / 2
            
            # Calculate the Asymmetry Index (AI)
            AI = abs(5 - IPRC)
            
            # Calculate the IPRAS for the item
            IPRAS = 2.35 + (1.5 * AI)
            
            # Calculate the IPR
            IPR = IPRU - IPRL
            
            # Calculate the Disagreement Index (DI)
            DI = IPR / IPRAS
            
            # Determine whether disagreement exists
            disagreement = DI > 1
            
            # Return all calculated values and the disagreement result
            return {
                "Median": median,
                "Q25": q25,
                "Q75":q75,
                "IQR": iqr,
                "IPRL": IPRL,
                "IPRU": IPRU,
                "IPRC": IPRC,
                "AI": AI,
                "IPRAS": IPRAS,
                "IPR": IPR,
                "DI": DI,
                "disagreement": disagreement
            }
        # Calculate disagreement index and print out measures
        results = calculate_disagreement_index(subset_nonzero)
        
        for key, value in results.items():
            print(f"{key}: {value}")
        
        # Determine text color based on background brightness
        if results['disagreement'] == False:
            color = color
        else:
            color = 'pink'

        # Assign text color of median box based on luminance of color
        text_color = 'black' if is_light_color(color) else 'white'
        
        # Calculate center position of x-axis
        x_center = max_count / 2
        
        # Create label for median value
        axes[i].text(x_center, 10.5, f"{median:.1f}", fontsize=18.5, color=text_color,
                     ha='center', va='center', bbox=dict(facecolor=color, edgecolor='none', pad = 20))

        axes[i].set_xlim(0, max_count)


    # Add a single "Median" label on the leftmost side
    fig.text(0.075, 0.05, "Median", va='center', ha='center', fontsize=18.5, rotation='horizontal', color='black')

    # Add carriage return for longer plot titles
    if len(cleaned_text) > 125:
        cleaned_text = cleaned_text[:96] + '\n' + cleaned_text[96:]
        fig.suptitle(f"{cleaned_text}",  fontsize=21, y=0.995)
    else:
        fig.suptitle(f"{cleaned_text}",  fontsize=21, y=0.975)
    plt.subplots_adjust(wspace=0.1)
    plt.show()
    return fig


# Function to process Excel file and generate separate PDFs
def process_excel_and_generate_plots(file_path, output_dir="[Output directory to send plots to]"):
    # Read Excel file
    df = pd.read_excel(file_path)

    # Get the version number for this run
    version = get_next_version(output_dir=output_dir)
    
    # Group data into chunks of 27 rows (3 Poll IDs x 9 rows each)
    num_rows_per_set = 27
    num_sets = len(df) // num_rows_per_set

    # Create a list to store all generated PDF paths
    generated_pdfs = []

    # Iterate through data and generate plots
    for i in range(num_sets):
        start_idx = i * num_rows_per_set
        end_idx = start_idx + num_rows_per_set
        # Subset the poll results data by three-question sets
        subset_df = df.iloc[start_idx:end_idx]

        # Generate the PDF path using the consistent version number
        output_pdf_path = os.path.join(output_dir, f"version_{version}_plot{i+1}.pdf")
        generated_pdfs.append(output_pdf_path)

        # Generate the plot for the current set
        fig = plot_poll_results(subset_df, title_prefix=f"Poll Set {i+1}")
        
        # Save the individual plot to PDF
        with PdfPages(output_pdf_path) as pdf:
            pdf.savefig(fig)
        plt.close(fig)    # Close the figure to free memory

    print(f"Generated PDFs for version {version}:")
    for pdf_path in generated_pdfs:
        print(f"- {pdf_path}")


# File paths
file_path = "[Filepath for Slido excel file]"  # Path to the input Excel file

# Generate and save plots
process_excel_and_generate_plots(file_path)